<h2>Find coordinates of census track centers and add 'hover' data</h2>

In [1]:
from geojson_utils import point_in_multipolygon
from geojson_utils import centroid
import pandas as pd
import numpy as np
import json

In [26]:
data= pd.read_json('C:/Users/mskac/machineLearning/GasLeakConEd/data/original/2010 Census Tracts.geojson')

In [27]:
centers = pd.DataFrame(columns=['ct', 'boro', 'nta', 'centerLong', 'centerLat', 'area'])
geojson = data
for element in data['features']:
    box_str = str(element['geometry']['coordinates'][0])
    box_str = '{"type": "Polygon","coordinates": '+box_str+'}'
    box = json.loads(box_str)
    point = centroid(box)
    centers = centers.append({'polygon':element['geometry']['coordinates'], 'ct': element['properties']['ct2010'],'nta': element['properties']['ntaname'].lower(), 'boro': element['properties']['boro_name'].lower(), 'centerLong': point['coordinates'][0], 'centerLat': point['coordinates'][1], 'area': element['properties']['shape_area']}, ignore_index=True)

In [28]:
#centers.drop_duplicates(subset ="geoid",keep=False,inplace=True) 

In [29]:
centers.head()

,ct,boro,nta,centerLong,centerLat,area,polygon
0,000900,staten island,west new brighton-new brighton-st. george,-74.082856,40.642204,2497009.71359,"[[[[-74.07920577013245, 40.64343078374567], [-..."
1,009800,manhattan,turtle bay-east midtown,-73.967775,40.755788,1906016.35002,"[[[[-73.96432543478758, 40.756381530990915], [..."
2,010000,manhattan,turtle bay-east midtown,-73.971232,40.758066,1860938.37721,"[[[[-73.96802436915851, 40.759578140052824], [..."
3,010200,manhattan,midtown-midtown south,-73.974450,40.759424,1860992.70485,"[[[[-73.97124277307127, 40.76093641847906], [-..."
4,010400,manhattan,midtown-midtown south,-73.977672,40.760779,1864600.37503,"[[[[-73.97445730550224, 40.76229308352487], [-..."


In [30]:
for index, row in centers.iterrows():
    with urllib.request.urlopen("https://geo.fcc.gov/api/census/area?lat={0}&lon={1}&format=json".format(row['centerLat'], row['centerLong'])) as url:
        data = json.loads(url.read().decode())
        centers.loc[index, 'block_fips'] = data['results'][0]['block_fips']              
centers.head()                               

,ct,boro,nta,centerLong,centerLat,area,polygon,block_fips
0,000900,staten island,west new brighton-new brighton-st. george,-74.082856,40.642204,2497009.71359,"[[[[-74.07920577013245, 40.64343078374567], [-...",360850009001003
1,009800,manhattan,turtle bay-east midtown,-73.967775,40.755788,1906016.35002,"[[[[-73.96432543478758, 40.756381530990915], [...",360610098006001
2,010000,manhattan,turtle bay-east midtown,-73.971232,40.758066,1860938.37721,"[[[[-73.96802436915851, 40.759578140052824], [...",360610100001003
3,010200,manhattan,midtown-midtown south,-73.974450,40.759424,1860992.70485,"[[[[-73.97124277307127, 40.76093641847906], [-...",360610102001006
4,010400,manhattan,midtown-midtown south,-73.977672,40.760779,1864600.37503,"[[[[-73.97445730550224, 40.76229308352487], [-...",360610104001003


In [31]:
centers['geoid'] = centers.block_fips.str[:11]
centers.drop(['ct', 'block_fips'], axis = 1, inplace = True)
centers.head()

,boro,nta,centerLong,centerLat,area,polygon,geoid
0,staten island,west new brighton-new brighton-st. george,-74.082856,40.642204,2497009.71359,"[[[[-74.07920577013245, 40.64343078374567], [-...",36085000900
1,manhattan,turtle bay-east midtown,-73.967775,40.755788,1906016.35002,"[[[[-73.96432543478758, 40.756381530990915], [...",36061009800
2,manhattan,turtle bay-east midtown,-73.971232,40.758066,1860938.37721,"[[[[-73.96802436915851, 40.759578140052824], [...",36061010000
3,manhattan,midtown-midtown south,-73.974450,40.759424,1860992.70485,"[[[[-73.97124277307127, 40.76093641847906], [-...",36061010200
4,manhattan,midtown-midtown south,-73.977672,40.760779,1864600.37503,"[[[[-73.97445730550224, 40.76229308352487], [-...",36061010400


In [33]:
for index, row in centers.iterrows():
    centers.at[index,'hover']=row['boro']+'<br>'+row['nta']+'<br>'+str(row['geoid']) 
    

In [34]:
centers.head()

,boro,nta,centerLong,centerLat,area,polygon,geoid,hover
0,staten island,west new brighton-new brighton-st. george,-74.082856,40.642204,2497009.71359,"[[[[-74.07920577013245, 40.64343078374567], [-...",36085000900,staten island<br>west new brighton-new brighto...
1,manhattan,turtle bay-east midtown,-73.967775,40.755788,1906016.35002,"[[[[-73.96432543478758, 40.756381530990915], [...",36061009800,manhattan<br>turtle bay-east midtown<br>360610...
2,manhattan,turtle bay-east midtown,-73.971232,40.758066,1860938.37721,"[[[[-73.96802436915851, 40.759578140052824], [...",36061010000,manhattan<br>turtle bay-east midtown<br>360610...
3,manhattan,midtown-midtown south,-73.974450,40.759424,1860992.70485,"[[[[-73.97124277307127, 40.76093641847906], [-...",36061010200,manhattan<br>midtown-midtown south<br>36061010200
4,manhattan,midtown-midtown south,-73.977672,40.760779,1864600.37503,"[[[[-73.97445730550224, 40.76229308352487], [-...",36061010400,manhattan<br>midtown-midtown south<br>36061010400


In [35]:
centers.to_csv('C:/Users/mskac/machineLearning/GasLeakConEd/data/processed/geoid_with_centers.csv')